In [80]:
import numpy as np
import pandas as pd

from ast import literal_eval
import os, time

# Random Functions

In [81]:
RESULT_DIR = "final_results"
benchmark = "sqrt"

In [82]:
df = {}
functions = None
for f_name in os.listdir(RESULT_DIR):
    try:
        benchmark_name = f_name.split("_")[1]
        tool_name =  f_name.split("_")[2]
        if benchmark_name == benchmark:
            df[tool_name] = pd.read_csv(os.path.join(RESULT_DIR, f_name)).sort_values('index')
    except Exception as e:
        pass

In [85]:
df['volesti']['details'].apply(literal_eval).apply(lambda x: np.asarray([t['output'] for t in x]))

0     []
1     []
2     []
3     []
4     []
5     []
6     []
7     []
8     []
9     []
10    []
11    []
12    []
13    []
14    []
15    []
16    []
17    []
18    []
19    []
20    []
21    []
22    []
23    []
24    []
25    []
26    []
27    []
28    []
29    []
Name: details, dtype: object

In [86]:
df_res = pd.DataFrame({
    "Function": df['faza']['formula'],
})
for tool in sorted(df):
    df_res[tool.capitalize()+"_output"] = df[tool]['output'].fillna(df[tool]['error']).astype(str).str[:500]
    if tool == "volesti":
        df['volesti']['volumes'] = df['volesti']['details'].apply(literal_eval).apply(lambda x: np.asarray([t['output'] for t in x]))
        a = df['volesti']['volumes'].apply(lambda x: x.mean())
        b = df['volesti']['volumes'].apply(lambda x: x.std())
        df_res[tool.capitalize()+"_meanstd"] = list(np.vstack((a, b)).T)
    df_res[tool.capitalize()+"_time"] = df[tool]['time'].apply(lambda x: round(x, 2))

df_res.head() 

/tmp/ipykernel_195415/3760899871.py:8: RuntimeWarning: Mean of empty slice.
  a = df['volesti']['volumes'].apply(lambda x: x.mean())
/home/des/venv/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


,Function,Faza_output,Faza_time,Gubpi_output,Gubpi_time,Latte_output,Latte_time,Psi_output,Psi_time,Volesti_output,Volesti_meanstd,Volesti_time
0,(8.14*x**0)**(1/2),"(2.798497619628902, 2.898483886718747)",6.94,"Command '['./app/GuBPI', 'experimental_results...",0.27,N\S,0.0,E[p_] = (407/50)^(1/2),0.02,N\S,"[nan, nan]",0.0
1,(2.51*x**1+4.63*x**0)**(1/2),"(2.369177856445304, 2.468756103515617)",6.12,"(2.4207947638040466, 2.4217356529114653)",0.40,N\S,0.0,E[p_] = (357/50)^(3/2)*200/753+-(463/100)^(3/2...,0.04,N\S,"[nan, nan]",0.0
2,(6.74*x**2+2.58*x**1+8.33*x**0)**(1/2),"(3.370510101318358, 3.470426940917966)",11.53,"(2.979619319933553, 3.4244508837497207)",0.42,N\S,0.0,E[p_] = 1/Integrate[1/(-544801/2500+674/25*xi1...,1.58,N\S,"[nan, nan]",0.0
3,(5.7*x**3+9.91*x**2+2.4*x**1+2.86*x**0)**(1/2),"(2.786684455871587, 2.8864392089843802)",46.57,"(2.2496577364511694, 2.839232991838528)",0.45,N\S,0.0,E[p_] = 1/(-Integrate[Boole[-1+xi1<=0]*Boole[-...,0.04,N\S,"[nan, nan]",0.0
4,(2.6*x**1+8.12*x**0)**(1/2),"(3.016962890624993, 3.116906738281241)",7.80,"(3.065971707694609, 3.0675433627474873)",0.41,N\S,0.0,E[p_] = (268/25)^(3/2)*10/39+-(203/25)^(3/2)*1...,0.04,N\S,"[nan, nan]",0.0


In [84]:
df_res.to_csv(os.path.join(RESULT_DIR, f'result_{benchmark}_{int(time.time())}.csv'), index=False)